In [ ]:
import os,sys,psycopg2,ipywidgets;
from IPython.display import display;
from IPython.display import Javascript;
from contextlib import closing;

sys.path.append(os.path.join(os.path.expanduser('~'),'notebooks'));
import common;

dbse = os.environ['POSTGRESQL_DB'];
host = os.environ['POSTGRESQL_HOST'];
port = os.environ['POSTGRESQL_PORT'];
user = 'cipsrv_upload';
pswd = os.environ['POSTGRESQL_CIP_PASS'];
uptrg = os.path.join(os.path.expanduser('~'),'notebooks/upload');


In [ ]:
cs = "dbname=%s user=%s password=%s host=%s port=%s" % (
     dbse
    ,user
    ,pswd
    ,host
    ,port
);

try:
    conn = psycopg2.connect(cs);
    conn.set_session(autocommit=True);
except:
    raise Exception("database connection error");

print("database is ready");


In [ ]:
upload = ipywidgets.FileUpload(
     accept   = '.zip,.gpkg'
    ,multiple = False
);
output2 = ipywidgets.Output();
display(upload);

def on_value_change(change):
    print('name: ' + upload.value[0]['name']);
    print('size: ' + str(upload.value[0]['size']));
    print('last_modified: ' + str(upload.value[0]['last_modified']));
    display(Javascript('IPython.notebook.select_next()'));
    display(Javascript('IPython.notebook.focus_cell()'));
    
upload.observe(on_value_change,names='value');


In [ ]:
upfile = upload.value[0];
upstem = upfile['name'];
upstem = upstem.removesuffix('.zip');
upstem = upstem.removesuffix('.gpkg');
upstem = upstem.removesuffix("_cip");
upext  = upfile['name'][-3:];
uploc  = os.path.join(uptrg,upfile['name']);
print('stem: ' + upstem);
print('upload location: ' + uploc);

with open(
     uploc
    ,'wb'
) as output_file: 
    content = upfile['content'];
    output_file.write(content);
    

In [ ]:
with closing(conn.cursor()) as cursor:
    for item in ['_control','_points','_lines','_areas','_cip','_sfid']:
        cursor.execute("DROP TABLE IF EXISTS cipsrv_upload." + upstem + item + " CASCADE");
    

In [ ]:
if upext == 'zip':
    uptrg2 = '/vsizip/' + uptrg;
else:
    uptrg2 = uptrg;
    
for item in ['_control','_points','_lines','_areas']:
    cmd = '-overwrite -f PostgreSQL '                                                      \
        + 'postgresql://' + user + ':' + pswd + '@' + host + ':' + port + '/' + dbse + ' ' \
        + os.path.join(uptrg2,upfile['name']) + ' ' + upstem + item + ' '                  \
        + '-nln cipsrv_upload.' + upstem + item + ' ';
    
    if item != '_control':
        cmd = cmd                                                                          \
        + '-t_srs EPSG:4269 '                                                              \
        + '-lco GEOMETRY_NAME=shape '                                                      \
        + '-lco GEOM_TYPE=geometry '                                                       \
        + '-lco LAUNDER=YES '                                                              \
        + '-lco DIM=2 '                                                                    \
        + '-lco SPATIAL_INDEX=GIST '                                                       \
        + '-nlt GEOMETRY';

    z = common.ogr2ogr(
         cmdstring = cmd
    );
    print(str(z[1]));
    

In [ ]:
with closing(conn.cursor()) as cursor:
    for item in ['_control','_points','_lines','_areas']:
        cursor.execute("SELECT COUNT(*) FROM cipsrv_upload." + upstem + item + " ");
        result = cursor.fetchone();
        print(upstem + item + ": " + str(result[0]));
    

In [ ]:
conn.close();